In [1]:
%%bash
wget -nc -c https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
wget -nc -c https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.names

File ‘wdbc.data’ already there; not retrieving.

File ‘wdbc.names’ already there; not retrieving.



In [2]:
import pandas as pd
import numpy as np
names = ["radius", "texture", "perimeter", "area", "smoothness", "compactness", 
         "concavity", "concave points", "symmetry", "fractal dimension"]
final_names = ["ID", "Diag"] + [name+'_mean' for name in names] + [name+'_std' for name in names] + [name+'_worst' for name in names]
df = pd.read_csv('wdbc.data', index_col=0,names=final_names)
df['Diag'] = df['Diag'].map({'M': 0, 'B': 1})
df.head()

,Diag,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal dimension_worst
ID,,,,,,,,,,,,,,,,,,,,,
842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Regresión logística

- Para escribir menos llamemos $f(x_i, \theta) = \theta_0 + \sum_{j=1}^D \theta_j x_{ij}$
- Mis observaciones son $\{y_i\}$ que son binarias
- Mi modelo para $y_i$ es $\mathcal{S} (f(x_i, \theta))$
- Los parámetros del modelo son $\theta$
- Asumiendo iid tenemos que la verosimilitud conjunta es igual a la multiplicación de las marginales
- Aplicando además logaritmo tenemos

$$
\log \mathcal{L}(\theta) = \sum_{i=1}^M \log p(y_i | \mathcal{S} (f(x_i, \theta)) )
$$

- Si asumimos que la probabilidad de $y_i$ dado el modelo $\mathcal{S} (f(x_i, \theta))$ es Bernoulli entonces

$$
\begin{align}
\log p(y_i | \mathcal{S} (f(x_i, \theta) ) &= \log \mathcal{S} (f(x_i, \theta))^{y_i} (1-\mathcal{S} (f(x_i, \theta)) )^{1-y_i} \nonumber \\
&= y_i \log (\mathcal{S} (f(x_i, \theta)) ) + (1-y_i) \log(1-\mathcal{S} (f(x_i, \theta)) )
\end{align}
$$

y todo junto

$$
\log \mathcal{L}(\theta) = \sum_{i=1}^M y_i \log (\mathcal{S} (f(x_i, \theta)) ) + (1-y_i) \log(1-\mathcal{S} (f(x_i, \theta)) )
$$

y si derivada con respecto a $\theta_j$

$$
\frac{d}{d\theta_j} \log \mathcal{L}(\theta) = \sum_{i=1}^M \left ( \frac{y_i}{\mathcal{S} (f(x_i, \theta))} - \frac{1-y_i}{1-\mathcal{S} (f(x_i, \theta))} \right) \frac{d \mathcal{S} (f(x_i, \theta))}{d f(x_i, \theta)} \frac{d f(x_i, \theta)}{d \theta_j} 
$$

donde

$$
\frac{d \mathcal{S}(z)}{dz} =  \mathcal{S}(z) ( 1 - \mathcal{S}(z))
$$

entonces

$$
\begin{align}
\frac{d}{d\theta_j} \log \mathcal{L}(\theta) &= \sum_{i=1}^M \left( y_i - y_i\mathcal{S} (f(x_i, \theta)) - \mathcal{S} (f(x_i, \theta))  + y_i \mathcal{S} (f(x_i, \theta)) \right) \frac{d f(x_i, \theta)}{d \theta_j} \nonumber \\
&= \sum_{i=1}^M \left( y_i  - \mathcal{S} (f(x_i, \theta)) \right) \frac{d f(x_i, \theta)}{d \theta_j} \nonumber 
\end{align}
$$

y 

$$
\frac{d f(x_i, \theta)}{d \theta_0} =  1
$$

para $j>0$
$$
\frac{d f(x_i, \theta)}{d \theta_j} =  x_{ij}
$$

Podemos simplicar más la verosimilitud usando $\log(1-\mathcal{S} (f(x_i, \theta)) ) = \log \mathcal{S} (f(x_i, \theta)) - f(x_i, \theta)$

$$
\begin{align}
\log \mathcal{L}(\theta) &= \sum_{i=1}^M y_i \log (\mathcal{S} (f(x_i, \theta)) ) + (1-y_i) \log(1-\mathcal{S} (f(x_i, \theta)) ) \nonumber \\
&=\sum_{i=1}^M \log (\mathcal{S} (f(x_i, \theta)) ) - (1-y_i) f(x_i, \theta) \nonumber
\end{align}
$$

In [3]:
# Crear base de datos
y = df["Diag"].values
X = df.drop(columns=["Diag"]).values
# Estandarizar
X = (X - np.mean(X, axis=0))/np.std(X, axis=0)
# Partición estratificada
import sklearn.model_selection
sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, train_size=0.75)
train_idx, test_idx = next(sss.split(X, y))

# Modelo y optimización
def sigmoide(z):
    return 1.0/(1.0 + np.exp(-z))

def modelo(theta, X):
    f = theta[0] + np.sum(theta[1:]*X, axis=1)     
    return sigmoide(f), f

def logverosimilitud(theta, *args):
    X, y = args
    s, f = modelo(theta, X)
    # Le pongo un signo menos ya que quiero minimizar en lugar de maximizar
    return -np.sum(-np.logaddexp(0, -f) - (1-y)*f, axis=0) # Versión simplificada
    #return -np.sum(y*np.log(s+1e-10) + (1-y)*np.log(1-s+1e-10) )

def grad_logverosimilitud(theta, *args):
    X, y = args
    N = len(y)
    s, f = modelo(theta, X)
    X1 = np.concatenate((np.ones(shape=(N, 1)), X), axis=1)
    e = (y - s)
    return -np.sum(e[:, np.newaxis]*X1, axis=0)

def eval_model(theta):  
    global best_theta, best_logl
    logltrain = logverosimilitud(theta, *(X[train_idx, :], y[train_idx]))
    logltest = logverosimilitud(theta, *(X[test_idx, :], y[test_idx]))
    print("Train: %0.4f, Test: %0.4f" %(logltrain, logltest))   
    if logltest < best_logl: # Guardar el mejor modelo de test
        best_theta = theta
        best_logl = logltest

import scipy.optimize
theta = np.random.randn(1+X.shape[1])
best_theta = np.zeros(1+X.shape[1])
best_logl = np.inf
res = scipy.optimize.minimize(logverosimilitud, x0=theta, method='BFGS', callback=eval_model, tol=1e-1, 
                              jac=grad_logverosimilitud, args=(X[train_idx, :], y[train_idx]))

# Evaluación
import sklearn.metrics
display("Parametros", best_theta)
haty_train, _ = modelo(best_theta, X[train_idx, :])
haty_test, _ = modelo(best_theta, X[test_idx, :])
# Encontrando el umbral
T = np.linspace(0, 1, num=100)
f1 = np.array([sklearn.metrics.f1_score(y[test_idx], haty_test > t) for t in T])
# Resultados de clasificación
display("Los resultados de clasificación en entrenamiento")
print(sklearn.metrics.classification_report(y[train_idx], haty_train > T[np.argmax(f1)]))
display("Los resultados de clasificación en validación")
print(sklearn.metrics.classification_report(y[test_idx], haty_test > T[np.argmax(f1)]))
cm = sklearn.metrics.confusion_matrix(y[test_idx], haty_test > T[np.argmax(f1)])
for i in range(2):
    for j in range(2):
        print("Era {0} y predije {1}: {2}".format(i, j, cm[i, j]))

Train: 129.6141, Test: 22.6140
Train: 86.8590, Test: 16.5338
Train: 68.1650, Test: 14.9821
Train: 56.4627, Test: 15.5080
Train: 49.7665, Test: 22.2173
Train: 45.5641, Test: 19.6069
Train: 42.7212, Test: 18.2291
Train: 38.5362, Test: 16.0013
Train: 34.7006, Test: 13.4873
Train: 33.0691, Test: 13.0948
Train: 30.9437, Test: 10.9753
Train: 28.1261, Test: 9.2674
Train: 25.6230, Test: 8.8313
Train: 23.2264, Test: 10.2564
Train: 20.8871, Test: 9.6851
Train: 19.5138, Test: 10.0940
Train: 18.3154, Test: 15.3029
Train: 17.4100, Test: 18.7566
Train: 16.5428, Test: 23.5208
Train: 15.7536, Test: 29.6564
Train: 15.0195, Test: 37.7201
Train: 14.3276, Test: 46.2564
Train: 13.6855, Test: 54.1870
Train: 13.1582, Test: 60.7293
Train: 12.6570, Test: 66.6111
Train: 12.0152, Test: 73.1328
Train: 11.2224, Test: 80.7100
Train: 10.3463, Test: 92.2113
Train: 9.3815, Test: 112.2614
Train: 8.6464, Test: 132.7383
Train: 8.0693, Test: 151.3934
Train: 7.6516, Test: 169.9709
Train: 6.9730, Test: 203.9358
Train: 5.866

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


'Parametros'

array([ 1.01150536, -1.70344626, -0.04014629, -2.0222858 ,  1.75672809,
       -0.64158738,  2.26243012, -2.15971349, -0.43965547,  0.1917149 ,
        0.44550931, -5.29303115,  0.44579182,  0.83341857,  3.80484004,
       -0.39295127, -0.48901102,  0.77793136, -0.64711842,  1.04911165,
        0.706907  , -1.54031477, -1.6897125 , -2.20114909,  1.38495793,
       -0.70647458,  1.5608941 , -0.85380461, -0.51521024, -1.65336352,
       -1.47080948])

/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


'Los resultados de clasificación en entrenamiento'

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       159
           1       0.99      0.99      0.99       267

    accuracy                           0.99       426
   macro avg       0.98      0.98      0.98       426
weighted avg       0.99      0.99      0.99       426



'Los resultados de clasificación en validación'

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        53
           1       0.99      1.00      0.99        90

    accuracy                           0.99       143
   macro avg       0.99      0.99      0.99       143
weighted avg       0.99      0.99      0.99       143

Era 0 y predije 0: 52
Era 0 y predije 1: 1
Era 1 y predije 0: 0
Era 1 y predije 1: 90
